In [4]:

#pip install pydicom
#!pip install kaggle
#!kaggle competitions download -c siim-acr-pneumothorax-segmentation # TEST data
#!kaggle datasets download -d seesee/siim-train-test # Full dataset
# Update kaggle creds
#! nano /home/ec2-user/.kaggle/kaggle.json


In [37]:

SRC_FILES = '/home/ec2-user/SageMaker/seg_project/src'

import sys
import numpy as np 
import pandas as pd 
from glob import glob
import pydicom
import os
from tqdm import tqdm
from pathlib import Path
import cv2

sys.path.append(SRC_FILES)
from mask_functions import rle2mask, mask2rle
from preprocessing import PreprocessorImages

%load_ext autoreload
%autoreload 2

BUCKET = 'pneumothorax-segmentation-dateset'
DCM_TRAIN_DIR = '/home/ec2-user/SageMaker/seg_project/data/raw_data/siim/dicom-images-train'
DCM_TEST_DIR = '/home/ec2-user/SageMaker/seg_project/data/raw_data/siim/dicom-images-test'
META_DATA_PATH = '/home/ec2-user/SageMaker/seg_project/data/meta_data_siim.csv'


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# load raw data from s3. 
#!aws s3 cp s3://pneumothorax-segmentation-dateset/dicom-images /home/ec2-user/SageMaker/data/raw_data/dicom-images --recursive
#!aws s3 cp s3://pneumothorax-segmentation-dateset/dicom-images/train-rle.csv ./data/raw_data/train-rle.csv


In [32]:

# Get a list of all the train and test files
train_files = sorted(glob(f'{DCM_TRAIN_DIR}/*/*/*.dcm'))
test_files = sorted(glob(f'{DCM_TEST_DIR}/*/*/*.dcm'))
                        

In [33]:

print(f' Example file = {train_files[0]}')


 Example file = /home/ec2-user/SageMaker/seg_project/data/raw_data/siim/dicom-images-train/1.2.276.0.7230010.3.1.2.8323329.1000.1517875165.878026/1.2.276.0.7230010.3.1.3.8323329.1000.1517875165.878025/1.2.276.0.7230010.3.1.4.8323329.1000.1517875165.878027.dcm


In [38]:

preprocessor = PreprocessorImages(resize_dim=256)
preprocessor.run(train_files, test_files, )


  0%|          | 6/12089 [00:00<03:53, 51.74it/s]

Running on the training set..


  0%|          | 8/3205 [00:00<00:41, 76.98it/s]

Running on the test set...


100%|██████████| 3205/3205 [00:41<00:00, 77.79it/s]


In [40]:
# Let have a look at the meta data
meta_data = preprocessor.get_meta_data()
meta_data.shape


(15294, 8)

In [41]:
# Save the computed meta data for analysis
meta_data.to_csv(META_DATA_PATH)


In [318]:
# Save data to s3 
#!aws s3 cp ./data s3://pneumothorax-segmentation-dateset/data --recursive
